In [3]:
#상위 폴더 binary_load.py파일 불러오기 위해 경로 추가
import sys
sys.path.append('../')  # 상위 폴더(main_folder)의 경로를 추가

from binary_load import binary_load_dataset

In [24]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from imblearn.over_sampling import ADASYN

from scikeras.wrappers import KerasClassifier

from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers as Layer
from tensorflow.keras.metrics import Recall, Precision, Accuracy

In [5]:
csv_loca = "../../../../data/Star.csv"
TEST_SIZE = 0.2
VAL_SIZE = 0.2
RANDOM_STATE = 42

In [6]:
def modelling():
  # 모델 만들기
  # 뉴런의 개수는 input과 output 사이의 숫자를 넣으라고 한다. --> 사용, 왜냐하면 시간이 적게 걸려서
  # 뉴런의 개수는 input의 2/3 정도 넣라고 한다.
  # 뉴런의 개수는 input의 두배보다는 적게 넣라고 한다.
  model = Sequential([Layer.Dense(6, input_shape)])
  model.add(Layer.Dense())

In [11]:
X_train, X_val, X_test, y_train, y_val, y_test, X_train_ad, X_val_ad, X_test_ad, y_train_ad, y_val_ad, y_test_ad  = binary_load_dataset(csv_loca, TEST_SIZE, VAL_SIZE, RANDOM_STATE)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((11454, 8), (11454,), (2864, 8), (2864,), (3580, 8), (3580,))

In [14]:
X_train_ad.shape, y_train_ad.shape, X_val_ad.shape, y_val_ad.shape, X_test_ad.shape, y_test_ad.shape

((20711, 8), (20711,), (5178, 8), (5178,), (6473, 8), (6473,))

In [15]:
model2 = Sequential()
model2.add(Layer.Dense(256, activation='sigmoid'))
model2.add(Layer.Dense(256, activation='sigmoid'))
model2.add(Layer.Dense(1, activation='sigmoid'))

model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

results = model2.fit(X_train, y_train, batch_size = 64, epochs=30, validation_data=(X_val,y_val))

Epoch 1/30
179/179 [==============================] - 0s 2ms/step - loss: 0.3083 - accuracy: 0.9034 - val_loss: 0.2552 - val_accuracy: 0.9085
Epoch 2/30
179/179 [==============================] - 0s 1ms/step - loss: 0.1813 - accuracy: 0.9327 - val_loss: 0.1260 - val_accuracy: 0.9595
Epoch 3/30
179/179 [==============================] - 0s 1ms/step - loss: 0.1161 - accuracy: 0.9612 - val_loss: 0.1035 - val_accuracy: 0.9675
Epoch 4/30
179/179 [==============================] - 0s 1ms/step - loss: 0.1024 - accuracy: 0.9676 - val_loss: 0.0971 - val_accuracy: 0.9689
Epoch 5/30
179/179 [==============================] - 0s 1ms/step - loss: 0.0968 - accuracy: 0.9706 - val_loss: 0.1006 - val_accuracy: 0.9679
Epoch 6/30
179/179 [==============================] - 0s 1ms/step - loss: 0.0943 - accuracy: 0.9717 - val_loss: 0.0994 - val_accuracy: 0.9682
Epoch 7/30
179/179 [==============================] - 0s 1ms/step - loss: 0.0928 - accuracy: 0.9728 - val_loss: 0.0938 - val_accuracy: 0.9717
Epoch 

In [17]:
model2.evaluate(X_test,  y_test, verbose=2)

112/112 - 0s - loss: 0.0774 - accuracy: 0.9799 - 71ms/epoch - 637us/step


[0.07735904306173325, 0.9798882603645325]

In [18]:
y_test_pred = model2.predict(X_test)

y_test_pred

112/112 [==============================] - 0s 567us/step


array([[0.00198126],
       [0.04162319],
       [0.00348267],
       ...,
       [0.02693617],
       [0.00068501],
       [0.04364026]], dtype=float32)

In [19]:
y_test_pred = (y_test_pred > 0.5).astype(np.float32)

y_test_pred

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [20]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_test_pred)

cm = pd.DataFrame(cm)
cm.columns = ['Predicted 0', 'Predicted 1']
cm.rename(index={0: "True 0", 1: "True 1"}, inplace = True)
cm

,Predicted 0,Predicted 1
True 0,3231,21
True 1,51,277


In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      3252
           1       0.93      0.84      0.88       328

    accuracy                           0.98      3580
   macro avg       0.96      0.92      0.94      3580
weighted avg       0.98      0.98      0.98      3580



In [22]:
np.any(np.isnan(y_train))

False

In [46]:
tf.random.set_seed(7)

def model_builder(nodes=16, activation='relu'):

  model = Sequential()
  model.add(Layer.Dense(nodes, input_dim=8, activation=activation))
  model.add(Layer.Dense(nodes, activation=activation))
  model.add(Layer.Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

# keras.wrapper를 활용하여 분류기를 만듭니다
model = KerasClassifier(model=model_builder, batch_size=8, verbose=0)

# GridSearch
batch_size=[64, 128, 256]
epochs=[10, 20, 30]
nodes=[64, 128, 256]
activation=['sigmoid']
param_grid=dict(batch_size=batch_size, epochs=epochs, model__nodes=nodes, model__activation=activation)


# GridSearch CV를 만들기
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


In [27]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.9760782259472673 using {'batch_size': 64, 'epochs': 30, 'model__activation': 'sigmoid', 'model__nodes': 256}
Means: 0.9712764099877772, Stdev: 0.0016050965872926727 with: {'batch_size': 64, 'epochs': 10, 'model__activation': 'sigmoid', 'model__nodes': 64}
Means: 0.9717129387113673, Stdev: 0.0019007806056467057 with: {'batch_size': 64, 'epochs': 10, 'model__activation': 'sigmoid', 'model__nodes': 128}
Means: 0.9722367731796752, Stdev: 0.002352399787900706 with: {'batch_size': 64, 'epochs': 10, 'model__activation': 'sigmoid', 'model__nodes': 256}
Means: 0.9733717478610092, Stdev: 0.0012346896825328245 with: {'batch_size': 64, 'epochs': 20, 'model__activation': 'sigmoid', 'model__nodes': 64}
Means: 0.9746813340317794, Stdev: 0.0012890538729031912 with: {'batch_size': 64, 'epochs': 20, 'model__activation': 'sigmoid', 'model__nodes': 128}
Means: 0.9745067225423433, Stdev: 0.0019404671530376296 with: {'batch_size': 64, 'epochs': 20, 'model__activation': 'sigmoid', 'model__nodes': 256

In [28]:
model2 = Sequential()
model2.add(Layer.Dense(256, activation='sigmoid'))
model2.add(Layer.Dense(256, activation='sigmoid'))
model2.add(Layer.Dense(1, activation='sigmoid'))

model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

results = model2.fit(X_train_ad, y_train_ad, batch_size = 64, epochs=30, validation_data=(X_val_ad,y_val_ad))

Epoch 1/30
324/324 [==============================] - 1s 1ms/step - loss: 0.6231 - accuracy: 0.6384 - val_loss: 0.5126 - val_accuracy: 0.7710
Epoch 2/30
324/324 [==============================] - 0s 1ms/step - loss: 0.4864 - accuracy: 0.7702 - val_loss: 0.4363 - val_accuracy: 0.7899
Epoch 3/30
324/324 [==============================] - 0s 1ms/step - loss: 0.4341 - accuracy: 0.7913 - val_loss: 0.4173 - val_accuracy: 0.8019
Epoch 4/30
324/324 [==============================] - 0s 1ms/step - loss: 0.4193 - accuracy: 0.7983 - val_loss: 0.4047 - val_accuracy: 0.8048
Epoch 5/30
324/324 [==============================] - 0s 1ms/step - loss: 0.4108 - accuracy: 0.8034 - val_loss: 0.4005 - val_accuracy: 0.8069
Epoch 6/30
324/324 [==============================] - 0s 1ms/step - loss: 0.4111 - accuracy: 0.8038 - val_loss: 0.3921 - val_accuracy: 0.8098
Epoch 7/30
324/324 [==============================] - 0s 1ms/step - loss: 0.4001 - accuracy: 0.8085 - val_loss: 0.4134 - val_accuracy: 0.8044
Epoch 

In [38]:
model2.evaluate(X_test_ad,  y_test_ad, verbose=2)

203/203 - 0s - loss: 0.3710 - accuracy: 0.8223 - 133ms/epoch - 655us/step


[0.37096211314201355, 0.8223389387130737]

In [39]:
y_test_pred = model2.predict(X_test_ad)

y_test_pred

203/203 [==============================] - 0s 479us/step


array([[0.43864447],
       [0.96707976],
       [0.99918574],
       ...,
       [0.2100063 ],
       [0.11477936],
       [0.81111944]], dtype=float32)

In [40]:
y_test_pred = (y_test_pred > 0.5).astype(np.float32)

y_test_pred

array([[0.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [1.]], dtype=float32)

In [41]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test_ad, y_test_pred)

cm = pd.DataFrame(cm)
cm.columns = ['Predicted 0', 'Predicted 1']
cm.rename(index={0: "True 0", 1: "True 1"}, inplace = True)
cm

,Predicted 0,Predicted 1
True 0,2892,361
True 1,789,2431


In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_test_ad, y_test_pred))

              precision    recall  f1-score   support

           0       0.79      0.89      0.83      3253
           1       0.87      0.75      0.81      3220

    accuracy                           0.82      6473
   macro avg       0.83      0.82      0.82      6473
weighted avg       0.83      0.82      0.82      6473



In [44]:
tf.random.set_seed(7)

def model_builder(nodes=16, activation='relu'):

  model = Sequential()
  model.add(Layer.Dense(nodes, input_dim=8, activation=activation))
  model.add(Layer.Dense(nodes, activation=activation))
  model.add(Layer.Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

# keras.wrapper를 활용하여 분류기를 만듭니다
model = KerasClassifier(model=model_builder, batch_size=8, verbose=0)

# GridSearch
batch_size=[64, 128, 256]
epochs=[10, 20, 30]
nodes=[64, 128, 256]
activation=['sigmoid']
param_grid=dict(batch_size=batch_size, epochs=epochs, model__nodes=nodes, model__activation=activation)


# GridSearch CV를 만들기
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train_ad, y_train_ad)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


In [45]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8217855820547456 using {'batch_size': 64, 'epochs': 30, 'model__activation': 'sigmoid', 'model__nodes': 128}
Means: 0.8077350634379722, Stdev: 0.0028509628889968803 with: {'batch_size': 64, 'epochs': 10, 'model__activation': 'sigmoid', 'model__nodes': 64}
Means: 0.8085559765524217, Stdev: 0.00314377461445658 with: {'batch_size': 64, 'epochs': 10, 'model__activation': 'sigmoid', 'model__nodes': 128}
Means: 0.8100041184840957, Stdev: 0.0034125776080563435 with: {'batch_size': 64, 'epochs': 10, 'model__activation': 'sigmoid', 'model__nodes': 256}
Means: 0.8124669207755407, Stdev: 0.0024531191457031988 with: {'batch_size': 64, 'epochs': 20, 'model__activation': 'sigmoid', 'model__nodes': 64}
Means: 0.8122738589650428, Stdev: 0.004632047771267354 with: {'batch_size': 64, 'epochs': 20, 'model__activation': 'sigmoid', 'model__nodes': 128}
Means: 0.8134325515627437, Stdev: 0.004572679577153135 with: {'batch_size': 64, 'epochs': 20, 'model__activation': 'sigmoid', 'model__nodes': 256}
M

In [47]:
#ad(ADASYN 업샘플링 한 결과가 더 안좋음)